In [1]:
from operator import itemgetter
from utils import merge_utils as mu
from utils import write_utils as wu

In [2]:
def parse(sc, path):
    # Create rdd
    rdd = sc.textFile(path)
    # Select header to be removed
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
           .map(lambda x: x.split("\t"))
           .filter(lambda x: x[0] != '')
           .map(lambda x:(int(x[0]),setup_items(x[1])))
    )

    return rdd

def setup_items(items):
    item_rank = 0
    items_list = []
    items_split = items.split(',')
    
    for item in items_split:
        if item != '':
            item_rank += 1
            items_list.append((int(item),item_rank,0))
            
    return items_list

In [3]:
interaction_rdd = parse(sc, "../../Submissions/ens_subs/interactions_82k.csv")
impression_rdd = parse(sc, "../../Submissions/ens_subs/impressions_116k.csv")

cf_intint_rdd = parse(sc, "../../Submissions/ens_subs/cf_int_int_18.9k.csv")
cf_intimp_rdd = parse(sc, "../../Submissions/ens_subs/cf_int_imp_22.4k.csv")
cf_impint_rdd = parse(sc, "../../Submissions/ens_subs/imp_imp_int_int_33.2k_avg.csv")
cf_impimp_rdd = parse(sc, "../../Submissions/ens_subs/cf_imp_imp_31k.csv")
cf_i_intint_rdd = parse(sc, "../../Submissions/ens_subs/true_10k_CF_ITEM_greater_3_shr=1_knn=750_22k.csv")
cf_i_impimp_rdd = parse(sc, "../../Submissions/ens_subs/cf_item_imp_imp_20k.csv")

cb_idf_rdd = parse(sc, "../../Submissions/ens_subs/idf_45k.csv")
cb_concept_rdd = parse(sc, "../../Submissions/ens_subs/tf_ervin_37k.csv")

#baseline_rdd = parse(sc, "./submissions/")

In [6]:
print cf_intint_rdd.take(1)

[(2588672, [(1070094, 1, 0), (2277750, 2, 0), (767198, 3, 0), (2787255, 4, 0), (1761753, 5, 0), (1399856, 6, 0), (56993, 7, 0), (792737, 8, 0), (725713, 9, 0), (1244196, 10, 0), (1478178, 11, 0), (939069, 12, 0), (1051740, 13, 0), (2345241, 14, 0), (477856, 15, 0), (102583, 16, 0), (2493140, 17, 0), (2183898, 18, 0), (1293546, 19, 0), (850201, 20, 0), (1145139, 21, 0), (1486154, 22, 0), (1441157, 23, 0), (2265526, 24, 0), (2138076, 25, 0), (1705434, 26, 0), (414112, 27, 0), (612922, 28, 0), (2294453, 29, 0), (657183, 30, 0), (984924, 31, 0), (2187714, 32, 0), (964254, 33, 0), (1903969, 34, 0), (1271875, 35, 0), (2413494, 36, 0), (1092821, 37, 0), (1338724, 38, 0), (1582921, 39, 0), (689497, 40, 0)])]


In [19]:
def rank_avg(sc, rdds):
    ensemble_rdd = sc.emptyRDD()
    
    for rdd in rdds:
        ensemble_rdd = ensemble_rdd.union(rdd.map(lambda x: (x[0], [(i[0],i[1],1) for i in x[1]])))
    
    voting_rdd = (ensemble_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                     .map(lambda x: ((x[0],x[1][0]),(x[1][1],x[1][2])))
                     .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
                     .map(lambda x: (x[0][0],[(x[0][1],x[1][0]/x[1][1])]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=False)[:30]))
                     )
    
    return voting_rdd

In [22]:
recommendation_rdd = rank_avg(sc, [interaction_rdd,
                                impression_rdd,
                                cf_intint_rdd,
                                cf_intimp_rdd,
                                cf_impimp_rdd,
                                cf_impint_rdd,
                                cf_i_impimp_rdd,
                                cf_i_intint_rdd,
                                cb_idf_rdd,
                                cb_concept_rdd])

In [23]:
print recommendation_rdd.take(1)

[(76800, [(2293641, 1), (1063437, 1), (2224944, 1), (398809, 1), (2337484, 1), (1617570, 2), (862730, 2), (326608, 2), (880348, 3), (282769, 3), (671708, 3), (1994213, 3), (155380, 3), (474030, 4), (964497, 4), (1208167, 4), (1577023, 4), (1477419, 4), (2390058, 4), (2477853, 4), (1333841, 4), (1162250, 5), (1308041, 5), (1848945, 5), (1373867, 5), (1469617, 5), (2380717, 5), (2575832, 5), (2653086, 6), (784737, 6)])]


In [25]:
wu.prepareSubmission(recommendation_rdd,"results/rank_averaging")

Submission file results/rank_averaging created successfully
